In [96]:
import json
import pandas as pd
import yfinance as yf

In [131]:
ticker = '^GSPC'
start_date = '2001-01-01'
stop_date = '2021-01-01'
with open('summary.json', 'r') as f:
    summary = json.load(f)
# test_trade_signals = json.loads(summary['test_trade_signals'])
# test_trade_signals = pd.DataFrame(test_trade_signals)
# pred_trade_signals = json.loads(summary['pred_trade_signals'])
# pred_trade_signals = pd.DataFrame(pred_trade_signals).values.tolist()
# print(test_trade_signals)
test_trade_signals = pd.read_csv('test_trade_signals.csv', index_col=['Date']).drop(columns=['Unnamed: 0'])

history_data = yf.download(ticker, start=start_date, end=stop_date)
history_data['Date'] = history_data.index
# Convert history_data to Highcharts format
ohlc = history_data[['Date', 'Open', 'High', 'Low', 'Close']].values.tolist()
volume = history_data['Volume'].tolist()

[*********************100%***********************]  1 of 1 completed


In [132]:
history_data = history_data.drop(columns=['Date'])

In [133]:
test_trade_signals['Signal'].value_counts()

Sell    24
Buy     24
Name: Signal, dtype: int64

In [134]:
test_trade_signals.index = pd.to_datetime(test_trade_signals.index)

In [135]:
history_data.index[-1]

Timestamp('2020-12-31 00:00:00')

In [136]:
test_trade_signals.index[-1]

Timestamp('2020-12-31 00:00:00')

In [137]:
import numpy as np

In [138]:
for i in test_trade_signals.index:
    if test_trade_signals['Signal'][i] == 'Buy':
        test_trade_signals.loc[i, 'Buy'] = history_data['Close'][i]
    elif test_trade_signals['Signal'][i] == 'Sell':
        test_trade_signals.loc[i, 'Sell'] = history_data['Close'][i]

In [139]:
buy_signals = test_trade_signals['Buy'].dropna()
sell_signals = test_trade_signals['Sell'].dropna()


In [140]:
buy_signals.to_csv('buy_signals.csv')
sell_signals.to_csv('sell_signals.csv')

In [141]:
from django.http import JsonResponse


In [142]:
a = pd.read_csv('buy_signals.csv').values.tolist()
json.dumps(a)

'[["2015-04-07", 2076.330078125], ["2015-07-10", 2076.6201171875], ["2015-07-29", 2108.570068359375], ["2015-10-01", 1923.8199462890625], ["2016-02-12", 1864.780029296875], ["2016-05-25", 2090.5400390625], ["2016-07-07", 2097.89990234375], ["2016-10-06", 2160.77001953125], ["2016-11-07", 2131.52001953125], ["2017-04-19", 2338.169921875], ["2017-07-11", 2425.530029296875], ["2017-09-07", 2465.10009765625], ["2018-03-05", 2720.93994140625], ["2018-04-24", 2634.56005859375], ["2018-07-11", 2774.02001953125], ["2018-11-23", 2632.56005859375], ["2019-01-14", 2582.610107421875], ["2019-06-06", 2843.489990234375], ["2019-08-30", 2926.4599609375], ["2019-10-18", 2986.199951171875], ["2020-04-07", 2659.409912109375], ["2020-07-14", 3197.52001953125], ["2020-10-05", 3408.60009765625], ["2020-11-04", 3443.43994140625]]'

In [143]:
buy_signals = pd.read_csv('buy_signals.csv')
buy_signals['Date'] = pd.to_datetime(buy_signals['Date'])
buy_signals['Date'] = buy_signals['Date'].apply(lambda x: int(x.timestamp() * 1000))

buy_signals = buy_signals[['Date', 'Buy']].values.tolist()

buy_signals = json.dumps(buy_signals)
response = {
    'buy_signals': buy_signals,
}
response = json.dumps(response)

In [144]:
# merged_data = pd.merge(history_data, test_trade_signals, left_on='Date', right_index=True, how='left')
# merged_data
merged_data = pd.concat([history_data, test_trade_signals], axis=1, join='outer')
merged_data


,Open,High,Low,Close,Adj Close,Volume,Signal,Sell,Buy
Date,,,,,,,,,
2001-01-02,1320.280029,1320.280029,1276.050049,1283.270020,1283.270020,1129400000,NaN,NaN,NaN
2001-01-03,1283.270020,1347.760010,1274.619995,1347.560059,1347.560059,1880700000,NaN,NaN,NaN
2001-01-04,1347.560059,1350.239990,1329.140015,1333.339966,1333.339966,2131000000,NaN,NaN,NaN
2001-01-05,1333.339966,1334.770020,1294.949951,1298.349976,1298.349976,1430800000,NaN,NaN,NaN
2001-01-08,1298.349976,1298.349976,1276.290039,1295.859985,1295.859985,1115500000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2020-12-24,3694.030029,3703.820068,3689.320068,3703.060059,3703.060059,1883780000,NaN,NaN,NaN
2020-12-28,3723.030029,3740.510010,3723.030029,3735.360107,3735.360107,3535460000,NaN,NaN,NaN
2020-12-29,3750.010010,3756.120117,3723.310059,3727.040039,3727.040039,3393290000,NaN,NaN,NaN


In [145]:
merged_data.to_csv('merged_data.csv')

In [146]:
sell_signals = merged_data

In [147]:
sell_signals

,Open,High,Low,Close,Adj Close,Volume,Signal,Sell,Buy
Date,,,,,,,,,
2001-01-02,1320.280029,1320.280029,1276.050049,1283.270020,1283.270020,1129400000,NaN,NaN,NaN
2001-01-03,1283.270020,1347.760010,1274.619995,1347.560059,1347.560059,1880700000,NaN,NaN,NaN
2001-01-04,1347.560059,1350.239990,1329.140015,1333.339966,1333.339966,2131000000,NaN,NaN,NaN
2001-01-05,1333.339966,1334.770020,1294.949951,1298.349976,1298.349976,1430800000,NaN,NaN,NaN
2001-01-08,1298.349976,1298.349976,1276.290039,1295.859985,1295.859985,1115500000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2020-12-24,3694.030029,3703.820068,3689.320068,3703.060059,3703.060059,1883780000,NaN,NaN,NaN
2020-12-28,3723.030029,3740.510010,3723.030029,3735.360107,3735.360107,3535460000,NaN,NaN,NaN
2020-12-29,3750.010010,3756.120117,3723.310059,3727.040039,3727.040039,3393290000,NaN,NaN,NaN
